In [ ]:
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch==1.3.1

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [89]:
import torch

In [90]:
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [3]:
pip install gluonnlp

Note: you may need to restart the kernel to use updated packages.


In [92]:
import gluonnlp as nlp

In [93]:
import numpy as np

In [94]:
from tqdm import tqdm, tqdm_notebook

In [95]:
from kobert.utils import get_tokenizer

In [96]:
from kobert.pytorch_kobert import get_pytorch_kobert_model


In [97]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [98]:
##GPU 사용 시
device = torch.device('cuda')

In [100]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [50]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
# !wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2020-10-23 11:19:53--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
접속 www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... 접속됨.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2020-10-23 11:19:54--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc734d9a72305daebeb1bb85610e.dl.dropboxusercontent.com/cd/0/get/BBwRrvtGxDk8I0LZNdjK72V642M7T_lVAwbuD_1GESqdz1AOD_mrNcfzcZVrLlXIWLEJh5E3UiI8bY3Dp01X0biauWUg0v8T4Pu1yzkYlzXx9I7bzBqCrt2kKXgBtY9ji-E/file?dl=1# [following]
--2020-10-23 11:19:55--  https://uc734d9a72305daebeb1bb85610e.dl.dropboxusercontent.com/cd/0/get/BBwRrvtGxDk8I0LZNdjK72V642M7T_lVAwbuD_1GESqdz1AOD_mrNcfzcZVrLlXIWLEJh5E3UiI8bY3Dp01X0biauWUg0v8T4Pu1yzkYlzXx9I7

In [43]:
# !pip3 install pandas

  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 9.5MB 247kB/s eta 0:00:01    27% |█████████                       | 2.7MB 17.3MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/b8/e5/a64ef44a85397ba3c377f6be9c02f3cb3e18023f8c89850dd319e7945521/numpy-1.19.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4f/a4/879454d49688e2fad93e59d7d4efda580b783c745fd2ec2a3adf87b0808d/pytz-2020.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ee/ff/48bde5c0f013094d729fe4b0316ba2a24774b3ff1c52d924a8a4cb04078a/six-1.15.0-py2.py3-none-any.whl


In [101]:
import pandas as pd

In [86]:
data = pd.read_csv('./title_for_sentiment/total_test2.txt')

In [88]:
data

,Title,label
0,국민안전로봇 2021년까지 개발…'로봇물고기' 전철 피할까?,1
1,'국정원 해킹 의혹' 상임위 앞두고 與野 전초전,0
2,인터넷티비(TV)결합상품 인터넷가입 비교사이트 통해 통신사별 장단점 비교해야 유리,1
3,SKT LG KT 인터넷가입 비교사이트 ‘펭귄통신’ 설치 당일 현금지원 이벤트,1
4,설 연휴 고향 가는 길 언제 출발해야 덜 막힐까,1
...,...,...
491,"SKT 나만의 ARS 만들어주는 'T ARS' 출시""",1
492,SKT 2019년까지 11조원 투자,1
493,통신사 커버리지맵 정확도 SKT 1위 '오차 0%',1
494,"크리스마스 맞이 스마트폰 가격 인하…""연말 특수 기대",1


In [124]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   496 non-null    object
 1   label   496 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 7.9+ KB


In [102]:
dataset_train = nlp.data.TSVDataset("./ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [128]:
dataset_train


In [23]:
#dataset_test = nlp.data.TSVDataset('./title_for_sentiment/total_test2 - total_test(1).csv', num_discard_samples=1)

In [ ]:
#dataset_test = nlp.data.TSVDataset('./title_for_sentiment/test_dataset_V2.txt', field_indices=[1,2], num_discard_samples=1)

In [116]:
dataset_test = nlp.data.TSVDataset('./title_for_sentiment/test_dataset_V2.txt') # index 포함

In [142]:
dataset_test = nlp.data.TSVDataset('./title_for_sentiment/total_test2.txt') #utf-8

In [117]:
dataset_test

In [118]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [119]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [120]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [121]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)

In [122]:
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

IndexError: list index out of range

#리스트 범위 밖에 있는 인덱스를 가져오라고 해서 에러
#인코딩 이슈

In [61]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=1)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=1)
# train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size)
# test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size)

NameError: name 'data_test' is not defined

In [38]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [39]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

RuntimeError: PyTorch is not linked with support for opencl devices

In [40]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [41]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [42]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [43]:
scheduler = WarmupLinearSchedule(optimizer, num_warmup_steps=warmup_step, get_linear=t_total)

In [44]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [45]:
for e in range(num_epochs):
    
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    print("1")
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        
        
        
        optimizer.zero_grad()
        
        token_ids = token_ids.long().to(device)
        
        segment_ids = segment_ids.long().to(device)
        
        valid_length= valid_length
        label = label.long().to(device)
        
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        
        if batch_id % log_interval == 0:
            
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
        
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

1


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


BrokenPipeError: [Errno 32] Broken pipe